In [1]:
!ls ../data

JAXA_HRLULC_Japan_v23.12	    output_with_category_and_value.tif.aux.xml
JAXA_HRLULC_Japan_v23.12.zip	    polygonA.dbf
L03-b-14_5239-jgd_GML		    polygonA.prj
L03-b-14_5239-jgd_GML.zip	    polygonA.qix
LC_N35E139.tif			    polygonA.shp
output_with_category.tif	    polygonA.shx
output_with_category.tif.aux.xml    shizuoka
output_with_category_and_value.tif


In [7]:
import rasterio

# GeoTIFFファイルのパスを指定
tiff_file = "../data/LC_N35E139.tif"

# GeoTIFFを開く
with rasterio.open(tiff_file) as src:
    # メタデータを取得
    print("MetaData:", src.meta)
    
    # 画像データ（バンド情報）を読み込む
    band1 = src.read(1)  # 1番目のバンド
    
    # 座標変換情報を取得
    transform = src.transform
    print("Transform:", transform)
    
    # 座標系（CRS）を取得
    crs = src.crs
    print("CRS:", crs)

    bounds = src.bounds


MetaData: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 12000, 'height': 12000, 'count': 1, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(8.333333333334045e-05, 0.0, 139.0,
       0.0, -8.333333333334045e-05, 35.99999999999994)}
Transform: | 0.00, 0.00, 139.00|
| 0.00,-0.00, 36.00|
| 0.00, 0.00, 1.00|
CRS: EPSG:4326


In [6]:
band1[0][0]

np.uint8(9)

In [8]:
 bounds

BoundingBox(left=139.0, bottom=34.99999999999986, right=140.00000000000009, top=35.99999999999994)

In [3]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# 入力GeoTIFFファイル
input_tiff = "../data/LC_N35E139.tif"

# 出力GeoTIFFファイル
output_tiff = "../data/LC_N35E139_3857.tif"

# 変換する座標系（EPSG:3857）
dst_crs = "EPSG:3857"

# GeoTIFFを開く
with rasterio.open(input_tiff) as src:
    # リプロジェクションのパラメータを計算
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )

    print(f"transform: {transform}")
    
    # メタデータを更新
    kwargs = src.meta.copy()
    kwargs.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height
    })
    
    # 新しいGeoTIFFを保存
    with rasterio.open(output_tiff, "w", **kwargs) as dst:
        for i in range(1, src.count + 1):  # 各バンドをリプロジェクション
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest  # リサンプリング方法（ここではnearest）
            )

print("Reprojected GeoTIFF saved to:", output_tiff)


transform: | 10.39, 0.00, 15473409.22|
| 0.00,-10.39, 4300621.37|
| 0.00, 0.00, 1.00|
Reprojected GeoTIFF saved to: ../data/LC_N35E139_3857.tif


In [28]:
src = rasterio.open("../data/LC_N35E139.tif")

In [29]:
src.transform

Affine(8.333333333334045e-05, 0.0, 139.0,
       0.0, -8.333333333334045e-05, 35.99999999999994)

In [31]:
src = rasterio.open(tiff_file)

In [17]:
src.bounds

BoundingBox(left=15473409.220265027, bottom=4163879.2106042877, right=15584727.168571986, top=4300621.372044263)

In [18]:
src.transform

Affine(10.389952240709333, 0.0, 15473409.220265027,
       0.0, -10.389952240709333, 4300621.372044263)

In [20]:
src.crs

CRS.from_wkt('PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]')

In [40]:
import rasterio
from rasterio.windows import from_bounds

# GeoTIFFファイルのパス
tiff_file = "../data/LC_N35E139_3857.tif"

# 切り出したい緯度経度範囲（min_lon, min_lat, max_lon, max_lat）
#bounds = [4205495, 15537673, 4207209, 15538785]
# left, bottom, right, top
bounds = [15537673, 4205495, 15538785, 4207209]

# GeoTIFFを開く
with rasterio.open(tiff_file) as src:
    # 指定した緯度経度範囲をピクセル座標に変換
    window = from_bounds(bounds[0], bounds[1], bounds[2], bounds[3], src.transform)
    
    # 指定範囲のデータを読み込む（例: バンド1）
    data = src.read(1, window=window)  # バンド番号は1
    
    # 新しい変換行列を取得
    transform = src.window_transform(window)
    
    # メタデータを確認
    print("Metadata:", src.meta)
    print("Transform:", transform)

# 切り出したデータの形状を確認
print("Extracted data shape:", data.shape)


Metadata: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 10714, 'height': 13161, 'count': 1, 'crs': CRS.from_wkt('PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0 +lon_0=0 +x_0=0 +y_0=0 +k=1 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]'), 'transform': Affine(10.389952240709333, 0.0, 15473409.220265027,
       0.0, -10.389952240709333, 4300621.372044263)}
Transform: | 10.39, 0.00, 15537673.00|
| 0.00,-10.39, 4207209.00|
| 

In [37]:
window

Window(col_off=np.float64(6185.1852872988675), row_off=np.float64(8990.644988555403), width=np.float64(107.02647848962806), height=np.float64(164.96707206068095))

In [41]:
# 新しいGeoTIFFファイルのパス
output_tiff = "../data/LC_N35E139_3857_clip.tif"

# 切り出したデータを新しいGeoTIFFに保存
with rasterio.open(
    output_tiff,
    'w',
    driver='GTiff',
    height=data.shape[0],
    width=data.shape[1],
    count=1,  # バンド数
    dtype=data.dtype,
    crs=src.crs,
    transform=transform,
) as dst:
    dst.write(data, 1)  # バンド1を書き込み

print("Cropped GeoTIFF saved to:", output_tiff)


Cropped GeoTIFF saved to: ../data/LC_N35E139_3857_clip.tif


In [4]:
src = rasterio.open("../data/LC_N35E139_3857_clip.tif")

In [5]:
data1 = src.read(1)

In [6]:
src2 = rasterio.open("../data/LC_N35E139_3857_clip.tif")

In [7]:
data2 = src2.read(1)

In [8]:
diff = data1 - data2

In [1]:
import cv2

In [10]:
cv2.imwrite('../data/diff.png', diff)

True